# Introduksjon til sannsynlighet og statistikk
I denne notatboken skal vi leke med noen av begrepene vi tidligere har diskutert. Mange begreper fra sannsynlighet og statistikk er godt representert i store biblioteker for databehandling i Python, som `numpy` og `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Tilfeldige variable og fordelinger
La oss begynne med å trekke et utvalg på 30 verdier fra en uniform fordeling fra 0 til 9. Vi vil også beregne gjennomsnitt og varians.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

For å visuelt anslå hvor mange forskjellige verdier det er i prøven, kan vi plotte **histogrammet**:


In [ ]:
plt.hist(sample)
plt.show()

## Analyserer ekte data

Gjennomsnitt og varians er veldig viktige når man analyserer virkelige data. La oss laste inn data om baseballspillere fra [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Vi bruker en pakke kalt [**Pandas**](https://pandas.pydata.org/) her for dataanalyse. Vi vil snakke mer om Pandas og å jobbe med data i Python senere i dette kurset.

La oss beregne gjennomsnittsverdier for alder, høyde og vekt:


In [ ]:
df[['Age','Height','Weight']].mean()

La oss nå fokusere på høyde, og beregne standardavvik og varians:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

I tillegg til gjennomsnittet gir det mening å se på medianverdien og kvartilene. De kan visualiseres ved hjelp av en **boksplott**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Vi kan også lage boksdiagrammer av delmengder av datasettet vårt, for eksempel gruppert etter spillerrolle.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Merk**: Dette diagrammet antyder at gjennomsnittlig høyde for førstebaser er høyere enn høyden til andrebaser. Senere skal vi lære hvordan vi kan teste denne hypotesen mer formelt, og hvordan vi kan vise at dataene våre er statistisk signifikante for å påvise dette.  

Alder, høyde og vekt er alle kontinuerlige stokastiske variabler. Hva tror du fordelingen deres er? En god måte å finne det ut på er å tegne histogrammet av verdiene: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normalfordeling

La oss lage et kunstig utvalg av vekter som følger en normalfordeling med samme gjennomsnitt og varians som våre virkelige data:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Siden de fleste verdier i det virkelige liv er normalfordelte, bør vi ikke bruke en uniform tilfeldig tallgenerator til å generere prøve-data. Her er hva som skjer hvis vi prøver å generere vekter med en uniform fordeling (generert av `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Konfidensintervaller

La oss nå beregne konfidensintervaller for vektene og høydene til baseballspillere. Vi vil bruke koden [fra denne stackoverflow-diskusjonen](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hypotesetesting

La oss utforske forskjellige roller i vårt baseballspiller-datasett:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

La oss teste hypotesen om at førstebaser er høyere enn andrebaser. Den enkleste måten å gjøre dette på er å teste konfidensintervallene:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Vi kan se at intervallene ikke overlapper.

En statistisk mer korrekt måte å bevise hypotesen på er å bruke en **Student t-test**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

De to verdiene som returneres av `ttest_ind`-funksjonen er:
* p-verdien kan betraktes som sannsynligheten for at to fordelinger har samme gjennomsnitt. I vårt tilfelle er den veldig lav, noe som betyr at det er sterkt bevis som understøtter at førstebaserne er høyere.
* t-verdien er den mellomliggende verdien av normalisert gjennomsnittsforskjell som brukes i t-testen, og den blir sammenlignet med en terskelverdi for en gitt konfidensverdi.


## Simulering av en normalfordeling med sentralgrenseteoremet

Den pseudo-tilfeldige generatoren i Python er designet for å gi oss en jevn fordeling. Hvis vi ønsker å lage en generator for normalfordeling, kan vi bruke sentralgrenseteoremet. For å få en normalt fordelt verdi vil vi bare beregne gjennomsnittet av et utvalg som er generert jevnt.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korrelasjon og Evil Baseball Corp

Korrelasjon gjør det mulig for oss å finne relasjoner mellom dataserier. I vårt lekeksempel later vi som om det finnes et ondsinnet baseball-selskap som betaler spillerne sine etter høyden deres – jo høyere spilleren er, desto mer penger får han/hun. Anta at det er en grunnlønn på $1000, og en ekstra bonus fra $0 til $100, avhengig av høyde. Vi vil ta de ekte spillerne fra MLB, og beregne deres tenkte lønninger:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

La oss nå beregne kovarians og korrelasjon for disse sekvensene. `np.cov` vil gi oss en såkalt **kovariansmatrise**, som er en utvidelse av kovarians til flere variabler. Elementet $M_{ij}$ i kovariansmatrisen $M$ er en korrelasjon mellom inngangsvariablene $X_i$ og $X_j$, og diagonale verdier $M_{ii}$ er variansen til $X_{i}$. På samme måte vil `np.corrcoef` gi oss **korrelasjonsmatrisen**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

En korrelasjon lik 1 betyr at det er en sterk **lineær sammenheng** mellom to variabler. Vi kan visuelt se den lineære sammenhengen ved å plotte én verdi mot den andre:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

La oss se hva som skjer hvis relasjonen ikke er lineær. Anta at selskapet vårt bestemte seg for å skjule den åpenbare lineære avhengigheten mellom høyder og lønninger, og introduserte noe ikke-lineær i formelen, som for eksempel `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

I dette tilfellet er korrelasjonen litt mindre, men den er fortsatt ganske høy. Nå, for å gjøre sammenhengen enda mindre åpenbar, kan vi ønske å legge til litt ekstra tilfeldighet ved å legge til en tilfeldig variabel til lønnen. La oss se hva som skjer:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Kan du gjette hvorfor prikkene stiller seg opp i vertikale linjer slik? 

Vi har observert korrelasjonen mellom et kunstig konstruert konsept som lønn og den observerte variabelen *høyde*. La oss også se om de to observerte variablene, som høyde og vekt, korrelerer:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Dessverre fikk vi ingen resultater – bare noen merkelige `nan`-verdier. Dette skyldes at noen av verdiene i serien vår er udefinerte, representert som `nan`, noe som gjør at resultatet av operasjonen også blir udefinert. Ved å se på matrisen kan vi se at `Weight` er den problematiske kolonnen, fordi selvkorrelasjonen mellom `Height`-verdiene er blitt beregnet.

> Dette eksemplet viser viktigheten av **datapreparering** og **rensing**. Uten riktige data kan vi ikke beregne noe.

La oss bruke `fillna`-metoden for å fylle inn de manglende verdiene, og beregne korrelasjonen:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Det er faktisk en korrelasjon, men ikke en så sterk en som i vårt kunstige eksempel. Hvis vi ser på spredningsdiagrammet av en verdi mot den andre, ville sammenhengen være mye mindre åpenbar:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Konklusjon

I denne notatboken har vi lært hvordan man utfører grunnleggende operasjoner på data for å beregne statistiske funksjoner. Vi vet nå hvordan vi kan bruke et solid apparat av matematikk og statistikk for å bevise noen hypoteser, og hvordan vi kan beregne konfidensintervaller for tilfeldige variabler gitt et datasett.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi jobber for nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på originalspråket skal betraktes som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår som følge av bruk av denne oversettelsen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
